# MuSc Zero-Shot Anomaly Detection Tutorial

This notebook demonstrates how to use the MuSc algorithm for zero-shot industrial anomaly detection.

**What you'll learn:**
- Loading and configuring the MuSc model
- Processing single images
- Visualizing anomaly heatmaps
- Batch processing multiple images
- Interpreting detection results

## Setup

First, make sure you're running this notebook from the MuSc directory and have installed the package:

```bash
cd MuSc
pip install -e .
```

In [ ]:
import os
import sys

# Ensure we're in the right directory
if os.path.basename(os.getcwd()) == 'examples':
    os.chdir('..')

# Add MuSc to path
sys.path.insert(0, os.getcwd())

import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 1. Load Configuration

The MuSc model is configured via a YAML file. Here we'll create a configuration programmatically.

In [ ]:
# Configuration dictionary
config = {
    "datasets": {
        "img_resize": 224,           # Input image size
        "dataset_name": "mvtec_ad",  # Dataset type (for benchmark evaluation)
        "class_name": "ALL",         # Category filter
        "divide_num": 1,             # Dataset splitting
        "data_path": "./data/",      # Dataset root
    },
    "device": 0 if torch.cuda.is_available() else "cpu",
    "models": {
        "backbone_name": "dinov2_vitb14",  # Vision transformer backbone
        "batch_size": 1,
        "feature_layers": [11],            # Transformer layers to use
        "pretrained": "openai",
        "r_list": [1],                     # LNAMD aggregation radii
    },
    "testing": {
        "output_dir": "output_notebook",
        "vis": False,
        "vis_type": "single_norm",
        "save_excel": False,
    },
    "thresholds": {
        "image_threshold": 9.0,    # Detection threshold (1.0-10.0)
        "overlay_threshold": 3.5,  # Visualization intensity
    },
}

print("Configuration:")
print(f"  Backbone: {config['models']['backbone_name']}")
print(f"  Image size: {config['datasets']['img_resize']}")
print(f"  Device: {config['device']}")

## 2. Initialize the Model

Loading the model for the first time will download pre-trained weights (~1-2 GB). This may take a few minutes.

In [ ]:
from models.musc import MuSc

print("Loading MuSc model (this may download weights on first run)...")
model = MuSc(config)
print(f"Model loaded on device: {model.device}")

## 3. Load and Preprocess an Image

Let's create a helper function to load and preprocess images for inference.

In [ ]:
def load_image(image_path, target_size=224):
    """
    Load and preprocess an image for MuSc inference.
    
    Args:
        image_path: Path to the image file
        target_size: Size to resize the image to
    
    Returns:
        original: Original image (BGR)
        tensor: Preprocessed tensor [1, 3, H, W]
    """
    # Load image
    original = cv2.imread(image_path)
    if original is None:
        raise ValueError(f"Could not load image: {image_path}")
    
    # Convert BGR to RGB and resize
    img_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (target_size, target_size))
    
    # Normalize to [0, 1] and convert to tensor
    img_norm = img_resized.astype(np.float32) / 255.0
    tensor = torch.tensor(img_norm).permute(2, 0, 1).unsqueeze(0)
    
    return original, tensor

def create_synthetic_image(size=224, has_defect=False):
    """
    Create a synthetic test image.
    
    Args:
        size: Image dimensions
        has_defect: If True, add a synthetic defect
    
    Returns:
        original: Image as numpy array (BGR)
        tensor: Preprocessed tensor
    """
    # Create a simple pattern (like a product surface)
    np.random.seed(42)
    img = np.ones((size, size, 3), dtype=np.uint8) * 180  # Gray background
    
    # Add some texture
    noise = np.random.randint(0, 20, (size, size, 3), dtype=np.uint8)
    img = cv2.add(img, noise)
    
    if has_defect:
        # Add a "defect" - a dark spot
        center = (size // 3, size // 2)
        cv2.circle(img, center, 15, (50, 50, 50), -1)
        cv2.circle(img, center, 20, (100, 100, 100), 2)
    
    # Convert to tensor
    img_norm = img.astype(np.float32) / 255.0
    tensor = torch.tensor(img_norm).permute(2, 0, 1).unsqueeze(0)
    
    return img, tensor

print("Helper functions defined.")

## 4. Run Inference on Synthetic Images

Let's test the model with synthetic "normal" and "defective" images.

In [ ]:
# Create test images
normal_img, normal_tensor = create_synthetic_image(has_defect=False)
defect_img, defect_tensor = create_synthetic_image(has_defect=True)

# Display the images
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(cv2.cvtColor(normal_img, cv2.COLOR_BGR2RGB))
axes[0].set_title("Normal Image")
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(defect_img, cv2.COLOR_BGR2RGB))
axes[1].set_title("Image with Defect")
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Run inference
print("Running inference on both images...")

# Process both images as a batch
batch_tensor = torch.cat([normal_tensor, defect_tensor], dim=0)
batch_tensor = batch_tensor.to(model.device)

with torch.no_grad():
    anomaly_maps = model.infer_on_images([batch_tensor])

print(f"Anomaly maps shape: {anomaly_maps.shape}")

# Get scores
normal_score = anomaly_maps[0].max()
defect_score = anomaly_maps[1].max()

print(f"\nResults:")
print(f"  Normal image max score: {normal_score:.4f}")
print(f"  Defect image max score: {defect_score:.4f}")

## 5. Visualize Anomaly Heatmaps

Let's create a visualization function to display the anomaly maps.

In [ ]:
def visualize_anomaly(image, anomaly_map, title="Anomaly Detection", threshold=0.9):
    """
    Visualize an anomaly map overlaid on the original image.
    
    Args:
        image: Original image (BGR numpy array)
        anomaly_map: 2D anomaly scores [H, W]
        title: Plot title
        threshold: Score threshold for anomaly classification
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original")
    axes[0].axis('off')
    
    # Anomaly heatmap
    heatmap = cv2.resize(anomaly_map, (image.shape[1], image.shape[0]))
    im = axes[1].imshow(heatmap, cmap='jet', vmin=0, vmax=1)
    axes[1].set_title(f"Anomaly Map (max: {anomaly_map.max():.3f})")
    axes[1].axis('off')
    plt.colorbar(im, ax=axes[1], fraction=0.046)
    
    # Overlay
    heatmap_colored = cv2.applyColorMap((heatmap * 255).astype(np.uint8), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(image, 0.6, heatmap_colored, 0.4, 0)
    axes[2].imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    
    # Add detection result
    max_score = anomaly_map.max()
    status = "ANOMALY" if max_score > threshold else "OK"
    color = "red" if max_score > threshold else "green"
    axes[2].set_title(f"Overlay - {status}", color=color)
    axes[2].axis('off')
    
    fig.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

# Visualize results
visualize_anomaly(normal_img, anomaly_maps[0], "Normal Sample")
visualize_anomaly(defect_img, anomaly_maps[1], "Defective Sample")

## 6. Processing Real Images

If you have real images to test, place them in the `sample_data/` directory and run:

In [ ]:
# Example: Process images from sample_data directory
sample_dir = "sample_data"

if os.path.exists(sample_dir):
    # Find all images
    image_files = []
    for root, dirs, files in os.walk(sample_dir):
        for f in files:
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                image_files.append(os.path.join(root, f))
    
    print(f"Found {len(image_files)} images in {sample_dir}")
    
    # Process first few images
    for img_path in image_files[:5]:
        try:
            original, tensor = load_image(img_path, config['datasets']['img_resize'])
            tensor = tensor.to(model.device)
            
            with torch.no_grad():
                anomaly_map = model.infer_on_images([tensor])
            
            visualize_anomaly(original, anomaly_map[0], os.path.basename(img_path))
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
else:
    print(f"No {sample_dir}/ directory found.")
    print("To test with real images:")
    print("  1. Create sample_data/ directory")
    print("  2. Add some test images")
    print("  3. Re-run this cell")

## 7. Understanding the Scores

The anomaly score is a value between 0 and 1:
- **0**: Perfectly normal
- **1**: Highly anomalous

The threshold determines what gets flagged as anomalous:
- **Higher threshold** (0.9-1.0): Fewer false positives, only obvious defects
- **Lower threshold** (0.5-0.7): More sensitive, may have false positives

In [ ]:
# Experiment with different thresholds
thresholds = [0.5, 0.7, 0.9, 0.95]

print("Classification results at different thresholds:")
print("=" * 50)

for thresh in thresholds:
    normal_result = "ANOMALY" if normal_score > thresh else "OK"
    defect_result = "ANOMALY" if defect_score > thresh else "OK"
    
    print(f"\nThreshold: {thresh}")
    print(f"  Normal image: {normal_result} (score: {normal_score:.4f})")
    print(f"  Defect image: {defect_result} (score: {defect_score:.4f})")

## 8. Using Different Backbone Models

MuSc supports multiple vision transformer backbones. Here's how to switch:

In [ ]:
# Available backbones (uncomment one to test)
available_backbones = [
    # Fast (for real-time)
    "vit_tiny_patch16_224.augreg_in21k",
    "dino_deitsmall16",
    
    # Balanced (recommended)
    "dinov2_vitb14",  # <- Default
    "ViT-B-16",
    
    # High accuracy (slower)
    "dinov2_vitl14",
    "ViT-L-14",
]

print("Available backbone models:")
for backbone in available_backbones:
    print(f"  - {backbone}")

print("\nTo use a different backbone, modify config['models']['backbone_name']")
print("and reinitialize the model.")

## 9. Exporting Results

Here's how to save detection results programmatically:

In [ ]:
import json
from datetime import datetime

def export_results(results, output_path):
    """
    Export detection results to JSON.
    
    Args:
        results: List of detection results
        output_path: Path to save JSON file
    """
    export_data = {
        "export_time": datetime.now().isoformat(),
        "model_config": {
            "backbone": config['models']['backbone_name'],
            "image_size": config['datasets']['img_resize'],
        },
        "results": results,
    }
    
    with open(output_path, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"Results exported to {output_path}")

# Example: Export our test results
results = [
    {
        "image": "normal_synthetic",
        "max_score": float(normal_score),
        "is_anomaly": bool(normal_score > 0.9),
    },
    {
        "image": "defect_synthetic",
        "max_score": float(defect_score),
        "is_anomaly": bool(defect_score > 0.9),
    },
]

export_results(results, "output_notebook/detection_results.json")

## Summary

In this notebook, you learned how to:

1. **Configure** the MuSc model with different settings
2. **Load** images and preprocess them for inference
3. **Run inference** to detect anomalies
4. **Visualize** anomaly heatmaps
5. **Interpret** detection scores
6. **Export** results for further analysis

### Next Steps

- Try the GUI: `musc-gui` for real-time detection
- Try the CLI: `python demo.py --input your_image.png`
- Read the documentation in `README.md` and `ARCHITECTURE.md`
- Explore different backbone models for your use case